## Proyecto: Turismo de los Alpes  - Grupo 34
Felipe Nuñez - 202021673
<br>
Jeronimo Vargas Rendon - 202113305
<br>
Juan Manuel Pérez - 202021827

In [12]:
# Importar librerias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
import re
import spacy
import stanza
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\JUAN M PEREZ
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\JUAN M PEREZ
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Cargar Datos

In [13]:
# Carga de datos
df = pd.read_csv('tipo1_entrenamiento_estudiantes.csv')

## 1. Entendimiento de los datos

In [14]:
# Visualización de los primeros registros
print(df.head())

# Total de registros
print("Total de registros:", df.shape[0])

                                              Review  Class
0  Nos alojamos en una casa alquilada en la ciuda...      4
1  La comida está bien, pero nada especial. Yo te...      3
2  En mi opinión, no es una como muchos usuarios ...      3
3  esta curiosa forma que asemeja una silla de mo...      4
4  Lo mejor era la limonada. Me gusto la comida d...      2
Total de registros: 7875


De entrada tenemos que se encuentran 7875 observaciones, y hay dos variables. La primera es la variable Review que es una cadena de cáracteres y hace referencia a el comentario de los viajeros. La segunda variable es Class, esta variable es nuestra variable objetivo pues esta representa la satisfaccion del viajero.

Analisis de Dimension de calidad de datos:
### - Unicidad:

In [16]:
#Contar duplicados
print("Duplicados:" , df.duplicated().sum())

Duplicados: 71


Hay 71 observaciones duplicadas, es necesario eliminar las filas repetidas y dejar solo una observación. Ya que dejarlas incrementaria los efectos de algunas palabras dada su repetición en el modelo.

In [17]:
df = df.drop_duplicates()
print(df.duplicated().sum())

0


### - Completitud:

In [18]:
# Verificación de valores faltantes
print(df.isnull().sum())

Review    0
Class     0
dtype: int64


No hay valores nulos los datos estan completos. Se cumple completitud.

### - Validez:

In [19]:
df["Class"].describe()

count    7804.000000
mean        3.500128
std         1.324219
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: Class, dtype: float64

In [20]:
df.dtypes

Review    object
Class      int64
dtype: object

Los datos cumplen con el formato esperado. La varible objetivo cumple con el rango esperado entre 1 y 5. Pues para Review se puede observar que es de tipo object el cual hace referencia a una cadena de textos.

### - Consistencia:
Si cumple

## 2. Preparación de los datos

### Limpieza de los datos:

Para este proceso de limpieza de datos es necesario hacer una limpieza de las palabras que no agregan valor. Para esto se definio la función remove_ noise que se encarga de eliminar ciertos tipos de ruido o elementos no deseados del texto antes de continuar con el proceso de limpieza.

1. re.sub(r'<.*?>', '', text): Utiliza expresiones regulares para eliminar todas las etiquetas HTML presentes en el texto. Esto es importante si estás trabajando con texto extraído de páginas web o documentos HTML, ya que las etiquetas HTML no proporcionan información útil para el análisis de texto y pueden interferir con los procesos de limpieza y análisis posteriores.

2. re.sub(r'[0-9]+', '', text): Utiliza expresiones regulares para eliminar todos los números del texto. Al observar algunas de los Review muchos de los números que se encontraban, hacen referencia a fechas o horas. Por esta razón, consideramos que los numeros no agregaban valor a estos comentarios y se eliminaron los números de los comentarios.

3. re.sub(r'[^a-záéíóúñA-ZÁÉÍÓÚÑ\s]', '', text): Utiliza expresiones regulares para eliminar todos los caracteres especiales excepto los espacios en blanco y los caracteres alfabéticos (incluyendo letras con tilde y la ñ). Esto es útil para eliminar signos de puntuación, símbolos y otros caracteres que no aportan significado semántico al texto. Sin embargo, conserva los espacios en blanco para preservar la estructura del texto.

4. re.sub(r'\s+', ' ', text): Utiliza expresiones regulares para reemplazar múltiples espacios en blanco consecutivos por un solo espacio en blanco. Esto ayuda a normalizar la cantidad de espacios en el texto, lo que facilita el procesamiento posterior.

5. text = text.lower(): Convierte todo el texto a minúsculas. Esto es importante para garantizar la consistencia en el texto y evitar problemas de diferenciación entre mayúsculas y minúsculas durante el análisis.

La otra función que se definio es clean_text, que se encarga de las "stopwords". Las "stopwords" son palabras que se filtran del texto durante el análisis de procesamiento del lenguaje natural (NLP) porque no aportan un significado contextual importante para el análisis. La función especificamente tokeniza el texto en palabras y luego filtra aquellas que no son stopwords ni caracteres alfabéticos, reconstruyendo el texto sin las stopwords.

In [21]:
def remove_noise(text):
    text = re.sub(r'<.*?>', '', text)  # Eliminar HTML tags
    text = re.sub(r'[0-9]+', '', text) # Elimnar numeros
    text = re.sub(r'[^a-záéíóúñA-ZÁÉÍÓÚÑ\s]', '', text)  # Eliminar caracteres especiales
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

# Definición de las stopwords en español
stop_words = set(stopwords.words('spanish'))

def clean_text(text):
    words = word_tokenize(text)  # Tokenizar el texto en palabras
    filtered_words = [word for word in words if word not in stop_words and word.isalpha()]
    # Reconstrucción del texto
    return " ".join(filtered_words)

def preprocessing(text):
    cleaned_text = remove_noise(text)  # Remover ruido del texto
    cleaned_words = clean_text(cleaned_text)  # Limpiar el texto
    return cleaned_words

### Tokenización:

In [22]:
df['palabras']=df['Review'].apply(preprocessing)

df.head()

,Review,Class,palabras
0,Nos alojamos en una casa alquilada en la ciuda...,4,alojamos casa alquilada ciudad amurallada pare...
1,"La comida está bien, pero nada especial. Yo te...",3,comida bien especial mejor comida mexcan unido...
2,"En mi opinión, no es una como muchos usuarios ...",3,opinión usuarios reclaman gran paladar parece ...
3,esta curiosa forma que asemeja una silla de mo...,4,curiosa forma asemeja silla montar ahi nombre ...
4,Lo mejor era la limonada. Me gusto la comida d...,2,mejor limonada gusto comida mundo sosa frío


En la tokenización se observa que se aplica la funcion de preprocesamiento que se explico anteriormente. Como resultado de esta tokenización preprocesada, se obtienen únicamente las palabras que cuentan con un sentido semántico. Esto significa que las palabras resultantes después del preprocesamiento son aquellas que tienen relevancia para el análisis de texto.

### Normalización:

El objetivo principal de la lematización es reducir las palabras a su forma canónica para facilitar el análisis semántico. Para esto se utilizo la libreria de Stanza desarrollada por el grupo de investigación de la Universidad de Stanford. En la siguiente función, Stanza realiza la lematización de las palabras en español. Después de procesar el texto, Stanza identifica y reemplaza cada palabra con su lema correspondiente. 

In [ ]:
nlp = spacy.load('es_core_news_sm')
def normalize_text(text):
    doc = nlp(text.lower())
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)
df['palabras'] = df['palabras'].apply(normalize_text)

In [23]:
stanza.download('es')

nlp = stanza.Pipeline(lang='es', processors='tokenize,lemma')

def process_text_stanza(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
    return lemmatized_text

df['palabras'] = df['palabras'].apply(process_text_stanza)

2024-04-05 06:37:43 INFO: Downloaded file to C:\Users\JUAN M PEREZ S\stanza_resources\resources.json
2024-04-05 06:37:43 INFO: Downloading default packages for language: es (Spanish) ...
2024-04-05 06:37:46 INFO: File exists: C:\Users\JUAN M PEREZ S\stanza_resources\es\default.zip
2024-04-05 06:37:52 INFO: Finished downloading models and saved to C:\Users\JUAN M PEREZ S\stanza_resources
2024-04-05 06:37:52 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-05 06:37:53 INFO: Downloaded file to C:\Users\JUAN M PEREZ S\stanza_resources\resources.json
2024-04-05 06:37:53 WARNING: Language es package default expects mwt, which has been added
2024-04-05 06:37:53 INFO: Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | ancora          |
| mwt       | ancora          |
| lemma     | ancora_nocharlm |

2024-04-05 06:37:53 INFO: Using device: cpu
2024-04-05 06:37:53 INFO: Loading: tokenize
2024-04-05 06:37:54 INFO: Loading: mwt
2024-04-05 06:37:54 INFO: Loading: lemma
2024-04-05 06:37:54 INFO: Done loading processors!


In [24]:
df

,Review,Class,palabras
0,Nos alojamos en una casa alquilada en la ciuda...,4,alojamo casa alquilada ciudad amurallada parec...
1,"La comida está bien, pero nada especial. Yo te...",3,comida bien especial mejor comida mexcan unido...
2,"En mi opinión, no es una como muchos usuarios ...",3,opinión usuario reclamar gran paladar parecer ...
3,esta curiosa forma que asemeja una silla de mo...,4,curioso forma asemejar silla montar ahi nombre...
4,Lo mejor era la limonada. Me gusto la comida d...,2,mejor limonada gusto comida mundo sosa frío
...,...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3,motivo estancia venir congreso medico hospedar...
7871,Es difícil revisar el castillo porque apenas p...,3,difícil revisar castillo apenas poder caminar ...
7872,Si vas a Mérida no puedes perderte de este lug...,5,si ir mérida poder perderte lugar nuevo sucurs...
7873,"Este imperdible sitio, que lleva el nombre del...",5,imperdible sitio llevar nombre conquistador jo...


### Transformación

In [25]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['palabras']).toarray()


In [ ]:
count = CountVectorizer()
X_count = count.fit_transform(df['palabras'])
print(X_count.shape)
X_count.toarray()[0]

In [ ]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(df['palabras'])
print(X_dummy.shape)
X_dummy.toarray()[0]

## 3. Modelo

In [ ]:
y=df['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Regression logistica
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

modelLR = LogisticRegression()
modelLR.fit(X_train, y_train)
y_pred = modelLR.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


In [ ]:
# Modelo multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

modelNB = MultinomialNB()
modelNB.fit(X_train, y_train)
y_pred = modelNB.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


In [ ]:
# Modelo Random Forest
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier()
modelRF.fit(X_train, y_train)
y_pred = modelRF.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")



## 4. Evaluacion

### Predicciones Archivo de Prueba

In [ ]:
# Predicciones con el modelo a los siguientes Datos usando el mejor modelo LR

df_pred = pd.read_csv('particion_prueba_estudiantes.csv')

df_p = df_pred.copy()

# Predecir con el modleo de Liner Regression
df_p['Review'] = df_p['Review'].apply(remove_noise)
df_p['Review'] = df_p['Review'].apply(clean_text)
df_p['Review'] = df_p['Review'].apply(normalize_text)
df_p['Review'] = df_p['Review'].apply(preprocess_text)
X_pred = vectorizer.transform(df_p['Review']).toarray()
y_pred = modelLR.predict(X_pred)
df_p['Class'] = y_pred

# Guardar predicciones
df_pred['Class'] = y_pred
df_pred.to_csv('predicciones_estudiantes.csv', index=False)
print(df_p.head())





## 5. Resultados